In [ ]:
import os
import json
import numpy as np
import skimage as sk
import skimage.io as skio
import imageio
from matplotlib import pyplot as plt
from scipy.spatial import Delaunay
from scipy.interpolate import griddata
from scipy import ndimage

### Load images

In [ ]:
im1 = sk.img_as_float(plt.imread('photos/thomas.jpg')/255)
im2 = sk.img_as_float(plt.imread('photos/kanye.jpg')/255)

### Part 1: Triangulation

In [ ]:
# image_num is either 1 or 2
def triangulate(image_num, outfile):
    curr_im = eval(f'im{image_num}')

    # FOR .JSON FILES
    # Compute triangulation for image via Delaunay function
    with open('points/thomas_kanye.json') as f:
        correspondences = json.load(f)
        points = np.array(correspondences[f'im{image_num}Points'])
        tri = Delaunay(points)

    # Save image
    # plt.imshow(curr_im)
    # plt.triplot(points[:,0], points[:,1], tri.simplices)
    # plt.plot(points[:,0], points[:,1], 'o', markersize=2)
    # plt.axis('off')
    # plt.tight_layout(pad=0)
    # plt.savefig(outfile, bbox_inches='tight', pad_inches=0)
    # plt.close()
    # plt.show()

    return points, tri

im1_points, tri = triangulate(1, 'part1_results/triangulated_thomas.jpg')
im2_points, _ = triangulate(2, 'part1_results/triangulated_kanye.jpg')

### Part 2: Computing the "mid-way" face

In [ ]:
average_points = 0.5 * (im1_points + im2_points)

In [ ]:
def computeAffine(tri1_points, tri2_points):
    # Add ones to create homogeneous coordinates for the points
    A = np.vstack((tri1_points.T, np.ones((1, 3))))
    B = np.vstack((tri2_points.T, np.ones((1, 3))))
    return np.linalg.solve(A.T, B.T).T

In [ ]:
# Warp function to warp one image to the midway shape
def warp(im, im_points, average_points, tri):
    num_triangles = len(tri.simplices)
    warped_im = np.zeros_like(im)

    # Loop over each triangle
    for t in range(num_triangles):
        # Get the current triangle from both source and target
        curr_triangle = tri.simplices[t]
        curr_avg_triangle = tri.simplices[t]

        # Compute the affine transformation from source to average shape
        T = computeAffine(im_points[curr_triangle], average_points[curr_avg_triangle])

        # Create mask for the target triangle
        avg_triangle_points = average_points[curr_avg_triangle]
        rr, cc = sk.draw.polygon(avg_triangle_points[:, 1], avg_triangle_points[:, 0], im.shape)

        # Create a set of points inside the triangle
        target_points = np.vstack((cc, rr)).T
        target_points_homog = np.hstack((target_points, np.ones((len(target_points), 1))))

        # Apply the inverse transformation to map target points to source image
        T_inv = np.linalg.inv(T)
        source_points = (T_inv @ target_points_homog.T).T[:, :2]

        # Interpolate pixel values from the source image for each color channel
        row_coords, col_coords = source_points[:, 1], source_points[:, 0]
        coords = [row_coords, col_coords]
        for c in range(im.shape[2]):
            warped_im[rr, cc, c] = ndimage.map_coordinates(im[:, :, c], coords, order=1)

    return warped_im

### Part 3: The morph sequence

In [ ]:
def morph(im1, im2, im1_pts, im2_pts, tri, warp_frac, dissolve_frac):
    average_points = warp_frac * im1_points + (1 - warp_frac) * im2_points
    warped_im1 = warp(im1, im1_points, average_points, tri)
    warped_im2 = warp(im2, im2_points, average_points, tri)
    mid_way = dissolve_frac * warped_im1 + (1 - dissolve_frac) * warped_im2

    # plt.imshow(mid_way)
    # plt.axis('off')
    # plt.tight_layout(pad=0)
    # plt.show()

    return mid_way

morphed_im = morph(im1, im2, im1_points, im2_points, tri, 0.5, 0.5)

#### Generate evolution and gif

In [ ]:
frames = []
num_frames = 45

for frame in range(num_frames):
    threshold = frame / (num_frames - 1)
    morphed_im = morph(im1, im2, im1_points, im2_points, tri, threshold, threshold)
    frames.append(morphed_im)

# Convert each frame to uint8
frames = [np.clip(frame, 0, 1) * 255 for frame in frames]
frames = [frame.astype(np.uint8) for frame in frames]

imageio.mimsave('part3_results/morphed_kanye_thomas.gif', frames)

# Create a 9x5 grid of plots
fig, axes = plt.subplots(9, 5, figsize=(13, 27))
fig.subplots_adjust(hspace=0.2, wspace=0.0)

for i, ax in enumerate(axes.flat):
    if i < len(frames):
        ax.imshow(frames[i])
        ax.axis('off')
        ax.set_title(f'Alpha: {i / (num_frames - 1):.2f}', fontsize=14)
    else:
        ax.axis('off')

# fig.tight_layout(pad=1)
# fig.savefig('part3_results/evolution_kanye_thomas.jpg')
# plt.show()

### Part 4: The "mean face" of a population

Pictures and images are from the [FEI Face database](https://fei.edu.br/~cet/facedatabase.html)

In [ ]:
# Get file names of images
target_directory = os.path.join(os.getcwd(), 'photos', 'smiling_men')
contents = os.listdir(target_directory)
filenames = [f for f in contents if os.path.isfile(os.path.join(target_directory, f))]
filenames_without_ext = [os.path.splitext(f)[0] for f in filenames] # Remove the extensions

In [ ]:
images = []

for file in filenames:
    im = sk.img_as_float(plt.imread(f'photos/smiling_men/{file}')/255)
    images.append(im)

height, width = images[0].shape[:2] # Get dimension of first image (all images have the same dimensions)

In [ ]:
# Triangulate all images
im_points = []

for points_file in filenames_without_ext:
    points = np.loadtxt(f'points/smiling_men_points/{points_file}.pts', comments=("version:", "n_points:", "{", "}"))
    
    # Define the corner points
    corners = np.array([[0, 0], [width - 1, 0], [0, height - 1], [width - 1, height - 1]])
    points = np.vstack((points, corners))
    
    tri = Delaunay(points)
    im_points.append(points)

men_average_points = (1 / len(im_points)) * sum(im_points)

In [ ]:
# Warp all images
warped_images = []

count = 4

for image, points in zip(images, im_points):
    warped_im = warp(image, points, men_average_points, tri)
    warped_images.append(warped_im)

# Find average image
average_smiling_man = (1 / len(warped_images)) * sum(warped_images)

# Save image
average_smiling_man = np.clip(average_smiling_man, 0, 1) * 255
average_smiling_man = average_smiling_man.astype(np.uint8)

# plt.imshow(average_smiling_man)
# plt.axis('off')
# plt.tight_layout(pad=0)
# plt.savefig('part4_results/average_smiling_man.jpg', bbox_inches='tight', pad_inches=0)
# plt.show()

In [ ]:
# Create a 2x4 grid
fig, axs = plt.subplots(2, 4, figsize=(10, 8))  # 2 rows and 4 columns
axs = axs.flatten()  # Flatten the 2D array of axes

# Iterate over the first four images and their corresponding warped images
for i, (image, warped_im) in enumerate(zip(images[:4], warped_images[:4])):
    # Display the original image in the first row
    axs[i].imshow(image)
    axs[i].axis('off')
    axs[i].set_title('Original Face')

    # Display the warped image in the second row
    axs[i + 4].imshow(warped_im)  # Place warped images in the second row
    axs[i + 4].axis('off')
    axs[i + 4].set_title('Warped Face')

# Adjust layout
plt.tight_layout(pad=0.5)
plt.savefig('part4_results/face_vs_warped.jpg', bbox_inches='tight', pad_inches=0.1)
plt.show()

#### My face (if I was average... and smiling)

In [ ]:
# # Crop and resize images

cropped_thomas = im1[50:-110, 80:-90]
# plt.imshow(cropped_thomas)
# plt.axis('off')
# plt.tight_layout(pad=0)
# plt.savefig('part4_results/resized_thomas.jpg', bbox_inches='tight', pad_inches=0)

import cv2 as cv
h, w = cropped_thomas.shape[:2]
resized_average = cv.resize(average_smiling_man, (w, h), interpolation = cv.INTER_AREA)
# plt.imshow(resized_average)
# plt.axis('off')
# plt.tight_layout(pad=0)
# plt.savefig('part4_results/resized_average.jpg', bbox_inches='tight', pad_inches=0)

In [ ]:
# Compute triangulation for image via Delaunay function
with open('points/resized_thomas_resized_average.json') as f:
    correspondences = json.load(f)
    points_thomas = np.array(correspondences[f'im1Points'])
    points_avg = np.array(correspondences[f'im2Points'])
    tri = Delaunay(points_thomas)

warped_thomas = warp(cropped_thomas, points_thomas, points_avg, tri)
warped_average = warp(resized_average, points_avg, points_thomas, tri)

# plt.imshow(warped_thomas)
# plt.axis('off')
# plt.tight_layout(pad=0)
# plt.savefig('part4_results/thomas_to_average.jpg', bbox_inches='tight', pad_inches=0)

# plt.imshow(warped_average)
# plt.axis('off')
# plt.tight_layout(pad=0)
# plt.savefig('part4_results/average_to_thomas.jpg', bbox_inches='tight', pad_inches=0)

### Part 5: Caricatures

In [ ]:
alpha = 1.25
caricature_points = alpha * (points_thomas - points_avg) + points_avg
caricature_im = warp(cropped_thomas, points_thomas, caricature_points, tri)

# plt.imshow(caricature_im)
# plt.axis('off')
# plt.tight_layout(pad=0)
# plt.savefig('part5_results/caricature_1dot25.jpg', bbox_inches='tight', pad_inches=0)

### Bells & Whistles: Music video

chopin, brahms, armstrong, sinatra, elvis, whitney, tupac, kanye

In [ ]:
video_directory = os.path.join(os.getcwd(), 'photos', 'bnw')
# artists = os.listdir(video_directory)
# filenames = [f for f in artists if os.path.isfile(os.path.join(video_directory, f))]
# filenames_without_ext = [os.path.splitext(f)[0] for f in filenames] # Remove the extensions

files = ['armstrong','sinatra','kanye','tupac','brahms','elvis','chopin','whitney']

output_size = (481, 392)

for filename in files:
    # Load the image
    img_path = os.path.join(video_directory, filename + '.jpg')
    img = skio.imread(img_path)

    # Resize the image
    resized_img = sk.transform.resize(img, output_size, anti_aliasing=True)

    # Save the resized image
    output_path = os.path.join(os.path.join(os.getcwd(), 'photos', 'bnw_scaled'), filename + '.jpg')
    skio.imsave(output_path, (resized_img * 255).astype('uint8'))

In [ ]:
im1 = sk.img_as_float(plt.imread(f'photos/bnw_scaled/chopin.jpg')/255)
im2 = sk.img_as_float(plt.imread(f'photos/bnw_scaled/brahms.jpg')/255)
im1_points, tri = triangulate(1, 'na')
im2_points, _ = triangulate(2, 'na')
morphed_im = morph(im1, im2, im1_points, im2_points, tri, 0.5, 0.5)
plt.imshow(morphed_im)

In [ ]:
video = []

def morph_ims(im1_file, im2_file, points_file):
    im1 = sk.img_as_float(plt.imread(f'photos/bnw_scaled/{im1_file}')/255)
    im2 = sk.img_as_float(plt.imread(f'photos/bnw_scaled/{im2_file}')/255)
    im1_points, tri = triangulate(1, 'na')
    im2_points, _ = triangulate(2, 'na')
    # morphed_im = morph(im1, im2, im1_points, im2_points, tri, 0.5, 0.5)

    frames = []
    num_frames = 45

    for frame in range(num_frames):
        threshold = frame / (num_frames - 1)
        morphed_im = morph(im1, im2, im1_points, im2_points, tri, threshold, threshold)
        frames.append(morphed_im)

    # Convert each frame to uint8
    frames = [np.clip(frame, 0, 1) * 255 for frame in frames]
    frames = [frame.astype(np.uint8) for frame in frames]

    return frames

video.extend(morph_ims('brahms.jpg', 'chopin.jpg', 'points/chopin_brahms.json'))
video.extend(morph_ims('armstrong.jpg', 'brahms.jpg', 'points/brahms_armstrong.json'))
video.extend(morph_ims('sinatra.jpg', 'armstrong.jpg', 'points/armstrong_sinatra.json'))
video.extend(morph_ims('elvis.jpg', 'sinatra.jpg', 'points/sinatra_elvis.json'))
video.extend(morph_ims('whitney.jpg', 'elvis.jpg', 'points/elvis_whitney.json'))
video.extend(morph_ims('tupac.jpg', 'whitney.jpg', 'points/whitney_tupac.json'))
video.extend(morph_ims('kanye.jpg', 'tupac.jpg', 'points/tupac_kanye.json'))

imageio.mimsave('bnw_results/morphed.gif', video)

In [ ]:
# Define video properties
height, width, layers = video[0].shape
fps = 30
output_file = 'bnw_results/morphed.mp4'
fourcc = cv.VideoWriter_fourcc(*'mp4v')

# Create video writer
out = cv.VideoWriter(output_file, fourcc, fps, (width, height))

# Write frames to video
for frame in video:
    # Convert frame from RGB to BGR to fix color channel interpretation
    if len(frame.shape) == 3:
        frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)
    elif len(frame.shape) == 2:
        frame = cv.cvtColor(frame, cv.COLOR_GRAY2BGR)
    out.write(frame)

# Release the video writer
out.release()